# Ejercicios Pair Programming - Módulo 2 Sprint 2
## Lección 2 - ETL 2.1
### Natalia García Lago y Noemi Azorín

En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.


En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import numpy as np
import ast

In [2]:
df_clima = pd.read_csv("../ficheros/ETL1.csv", index_col= 0)
df_clima.reset_index()

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",14,-1,15,-9999,rain,1,0,335,2,Papua New Guinea
1,1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",16,-1,13,-9999,rain,1,0,10,2,Papua New Guinea
2,2,9,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",18,-4,12,-9999,rain,3,0,40,2,Papua New Guinea
3,3,12,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",18,-4,12,-9999,rain,3,0,25,2,Papua New Guinea
4,4,15,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",14,-1,15,-9999,rain,3,0,340,2,Papua New Guinea
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,315,180,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",0,10,15,1008,snow,3,4,85,2,USA
316,316,183,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",0,10,16,1008,snow,3,5,50,2,USA
317,317,186,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",0,10,15,1008,snow,4,5,10,2,USA
318,318,189,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",0,10,15,1009,snow,4,5,0,2,USA


In [3]:
df_ataques = pd.read_csv("../ficheros/attacks_limpieza_completa.csv", index_col = 0)
df_ataques.head()

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.783730,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.783730,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.783730,-100.445882,...,1,0,0,1,0,0,3,7,3,8.403629
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,...,1,0,0,1,0,0,3,7,7,6.313925
4,2018,Provoked,mexico,27.0,Tiger shark,Jun,N,M,23.658512,-102.007710,...,1,0,0,1,0,0,2,7,4,6.313925


In [4]:
dic = {"USA": [-100.445882, 39.7837304], "Australia": [134.755,-24.7761086], 
        "South Africa" : [24.991639,-28.8166236], "New Zealand": [172.8344077,-41.5000831], 
        "Papua New Guinea": [144.2489081,-5.6816069] }

In [5]:
#Hacemos una consulta iterando para no tener que hacer una por cada key de nuestro dicc
#Y creamos la columna paía
#La concatenamos al df vacio anterior
#Diccionario : pais, longitud y latitud
def llamada(diccionario):

    df_union = pd.DataFrame()
    
    for k, v in dic.items():
        
        response = requests.get(url=f'http://www.7timer.info/bin/api.pl?lon={v[0]}&lat={v[1]}&product=meteo&output=json')

        df_meteo = pd.json_normalize(response.json(), "dataseries")

        df_meteo["country"] = k

        df_union = pd.concat([df_meteo, df_union], axis = 0)
    return df_union 
    

In [6]:
clima = llamada(dic)
type(clima)

pandas.core.frame.DataFrame

In [9]:
def limpiar(df):
    #Realizamos filtro de países que nos interesan
    lista_selec = ['usa', 'australia', 'papua new guinea', 'new zealand', "south africa"]
    df_filtrado = df[df["country"].isin(lista_selec)]

    #Casteamos el object que contiene diccionarios
    #df['wind_profile'] = df['wind_profile'].apply(ast.literal_eval)
    #df['rh_profile'] = df['rh_profile'].apply(ast.literal_eval)

    #Desempaquetamos la lista de diccionarios en columnas
    y = df['wind_profile'].apply(pd.Series)
    x = df['rh_profile'].apply(pd.Series)

    #Separamos de los diccionarios por k, v para quedarnos con la info que nos interesa

    for i in range(len(x.columns)):
        nombre_rh = "rh_"+ str(x[i].apply(pd.Series)["layer"][0])
        valores_rh = (x[i].apply(pd.Series)["rh"])

        df.insert(i, nombre_rh, valores_rh)
    
    for i in range(len(y.columns)):
        nombre_wind = "wind_"+ str(y[i].apply(pd.Series)["layer"][0])
        valores_wind = list(y[i].apply(pd.Series)["direction"])
        
        df.insert(i, nombre_wind, valores_wind)

    #Hacemos un groupby y la media
    df_groupby = df.groupby("country").mean()

    return df_groupby


In [10]:
limpiar(clima)

/tmp/ipykernel_1893/2811488265.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_groupby = df.groupby("country").mean()


,"wind_0 950mb\n0 950mb\n0 950mb\n0 950mb\n0 950mb\nName: layer, dtype: object","wind_0 900mb\n0 900mb\n0 900mb\n0 900mb\n0 900mb\nName: layer, dtype: object","wind_0 850mb\n0 850mb\n0 850mb\n0 850mb\n0 850mb\nName: layer, dtype: object","wind_0 800mb\n0 800mb\n0 800mb\n0 800mb\n0 800mb\nName: layer, dtype: object","wind_0 750mb\n0 750mb\n0 750mb\n0 750mb\n0 750mb\nName: layer, dtype: object","wind_0 700mb\n0 700mb\n0 700mb\n0 700mb\n0 700mb\nName: layer, dtype: object","wind_0 650mb\n0 650mb\n0 650mb\n0 650mb\n0 650mb\nName: layer, dtype: object","wind_0 600mb\n0 600mb\n0 600mb\n0 600mb\n0 600mb\nName: layer, dtype: object","wind_0 550mb\n0 550mb\n0 550mb\n0 550mb\n0 550mb\nName: layer, dtype: object","wind_0 500mb\n0 500mb\n0 500mb\n0 500mb\n0 500mb\nName: layer, dtype: object",...,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed
country,,,,,,,,,,,,,,,,,,,,,
Australia,103.43750,98.281250,101.250000,105.234375,109.296875,115.156250,122.500000,126.953125,132.265625,154.765625,...,-9999.0,-9999.0,-9999.0,32.500000,-0.062500,2.671875,1005.843750,1.578125,0.000000,2.921875
New Zealand,-1454.53125,151.171875,169.765625,184.687500,174.531250,168.437500,151.875000,160.156250,164.843750,160.390625,...,-9999.0,-9999.0,-9999.0,15.328125,6.718750,9.734375,1018.656250,2.328125,0.000000,1.890625
Papua New Guinea,-9999.00000,-9999.000000,-9999.000000,184.296875,189.375000,239.609375,181.484375,105.312500,92.968750,90.078125,...,-9999.0,-9999.0,-9999.0,15.109375,-0.250000,13.015625,-9999.000000,3.968750,0.000000,2.015625
South Africa,-9999.00000,-9999.000000,141.328125,187.265625,201.718750,197.812500,222.656250,201.718750,194.531250,192.812500,...,-9999.0,-9999.0,-9999.0,31.390625,0.968750,0.234375,1010.265625,0.640625,0.000000,3.234375
USA,-9999.00000,190.625000,192.500000,208.750000,232.031250,251.484375,252.421875,264.140625,255.390625,264.843750,...,-9999.0,-9999.0,-9999.0,3.484375,11.734375,9.156250,1012.703125,0.984375,-781.078125,2.421875


In [ ]:
def union (df1, df2):
    #Concatenamos los df
    df_final = pd.concat([df1, df2], axis = 0)

    #guardamos los df 
    df_final.to_csv('../ficheros/final_ataques_clima.csv')
    
    return df_final


In [ ]:
class Extraccion: 
    # primero tenemos que crear el constructor con las variables globales que usaremos. Están son variables que podrán cambiar cada vez que llamemos a la clase. 
    def __init__(self, lat, lon, ciudad):

        # definimos cada una de las variables. Recordamos que tenemos que usar el método .self para definirlas y que la clase entienda que las podremos usar en otros métodos de la clase. 
        self.lat = lat
        self.lon = lon
        self.ciudad = ciudad

    # definimos nuestro primer método. El que nos hará la llamada a la API
    # Si comparamos, veremos que la función es exactamente igual a la que hicimos en la clase anterior, solo que con unos pequeños cambios. En este caso incluimos el parámetro self
    def llamada_API(self, producto):
        
        # producto es una variable nueva, por lo que lo primero que tenemos que hacer es definirla
        self.producto = producto
    
        # hacemos la llamada  a la API
        # Tendremos que poner el self delante de long y lat ya que son variables definidas en el constructor. El self podríamos decir que es como el camino de baldosas amarillas que indica al método de donde tiene que sacar esas variables
        url = f'http://www.7timer.info/bin/api.pl?lon=-{self.lon}&lat={self.lat}&product={producto}&output=json'

        response = requests.get(url=url)
        codigo_estado = response.status_code
        razon_estado = response.reason
        if codigo_estado == 200:
            print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 402:
            print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 404:
            print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        else:
            print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)

        # convertimos los resultados en un dataframe: 
        df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
        return df
    
    # definimos un nuevo método, el de limpiar los datos obtenidos de la llamada a la API para el producto civil. 
    def limpiar_civil(self, df): 

        # lo primero que tenemos que hacer es crear la columna de fecha en el dataframe nuevo 
        # En este caso será la fecha del día de hoy. 
        hoy = datetime.now()
        hoy = datetime.strftime(hoy, '%Y-%m-%d')

        # creamos la nueva columna
        df["fecha"] = hoy

        # de nuevo usamos el self para crear las nuevas columnas en función de los parámetros pasados al definir la clase. Recordamos, el caminito de baldosas amarillas.
        df["latitud"] = self.lat
        df["longitud"] = self.lon
        df["ciudad"] = self.ciudad
        return df
    
    
    # creamos una clase para limpiar el resultado de la llamada a la API para el producto astro
    def limpiar_astro(self, df):

        #seleccionamos solo las columnas que nos interesan
        df = df[["seeing", "transparency", "timepoint"]]

        # creamos la columna de fecha: 
        hoy = datetime.now()
        hoy = datetime.strftime(hoy, '%Y-%m-%d')
        df["fecha"] = hoy


        # insertamos las columnas de la localidad
    
        df["ciudad"] = self.ciudad

        return df
    
    # de nuevo, hacemos lo mismo que en la lección anterior. 
    def juntar_dfs(self, df_completo, df_civil, df_visibilidad): 

        df_hoy = pd.merge(df_civil , df_visibilidad , on=['fecha', "timepoint", "ciudad"], how = "right")
        print("El df de hoy es ", df_hoy.columns)
        print("-----------------------------------------")
        print("El df de completo es ", df_completo.columns)
        # lo primero que hacemos es concatenar los dataframes con la información general. df_completo y df_civil
        df_completo = pd.concat([df_completo, df_hoy], axis = 0)

        
        # guardamos los datos
        df_completo.to_pickle('../files/datos_actualizados.pkl')
        df_completo.to_csv('../files/datos_actualizados.csv')

        return df_completo
    
    

    def chequear_datos(self, df): 
    
        print("Las columnas son:", "\n")
        print(list(df.columns))
        print("-----------------------------------------")

        print("Los tipos de datos que tenemos son:", "\n")
        print(df.dtypes)
        print("-----------------------------------------")

        print("El porcentaje de nulos:", "\n")
        print((df.isnull().sum() / df.shape[0]) *  100)

    def limpiar_dataframe(self, df, lista_columnas = []): 

        #convertimos la fecha a datetime
        df["fecha"] = pd.to_datetime(df["fecha"])

        # reemplazamos los nulos de las columnas por la media
         # lista de columnas en las que queremos reemplazar los nulos
        df[lista_columnas]=df[lista_columnas].fillna(df.mean())

        
        # renombrar columnas
        
        df.rename(columns = {"ciudad_x": "ciudad"}, inplace = True)

        # quitar % 
        df["rh2m"] = df["rh2m"].replace(r"%", "", regex = True)

        # guardamos los datos una vez limpios
        df.to_pickle('../files/datos_actualizados.pkl')
        df.to_csv('../files/datos_actualizados.csv')

        return df